In [1]:
import sqlite3
import pandas as pd

In [2]:
# Load the CSV file into a pandas DataFrame
csv_file = r"C:\Users\ah230\Downloads\DABC\Class Folder\New Project 3\bigfoot-sightings\data\state_population.csv"
state_population_df = pd.read_csv(csv_file)
# Keep only the 'NAME' and 'POPESTIMATE2023' columns from the state_population_df
state_population_df = state_population_df[['NAME', 'POPESTIMATE2023']]
state_population_df = pd.read_csv(csv_file, encoding='utf-8')  # or try 'latin1' if utf-8 fails

In [3]:
# Load the JSON file into a pandas DataFrame
json_file = r"C:\Users\ah230\Downloads\DABC\Class Folder\New Project 3\bigfoot-sightings\data\bigfoot_coordinates_clean_cols.json"
bigfoot_coordinates_df = pd.read_json(json_file)
bigfoot_coordinates_df = pd.read_json(json_file, encoding='utf-8')  # or try 'latin1' if utf-8 fails


In [4]:
# Merge the two DataFrames on 'state' from the JSON and 'STATE' from the CSV
merged_df = pd.merge(bigfoot_coordinates_df, state_population_df, left_on='state', right_on='NAME', how='inner')
merged_df = merged_df.dropna(subset=['state', 'latitude', 'longitude', 'POPESTIMATE2023'])
print(merged_df.isna().sum())

report_number       0
report_class        0
state               0
county              0
latitude            0
                   ..
RDOMESTICMIG2022    0
RDOMESTICMIG2023    0
RNETMIG2021         0
RNETMIG2022         0
RNETMIG2023         0
Length: 69, dtype: int64


In [5]:
# Display the first few rows of the merged dataframe
print(len(merged_df))

4745


In [6]:
# Connect to SQLite (this will create the database if it doesn't exist)
conn = sqlite3.connect('bigfoot_population.db', detect_types=sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES)
cursor = conn.cursor()

In [7]:
# Drop the table if it exists (this will delete the table and its data)
cursor.execute('DROP TABLE IF EXISTS bigfoot_population')

In [8]:
# Create the table to store the merged data
cursor.execute('''
    CREATE TABLE IF NOT EXISTS bigfoot_population (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        report_class VARCHAR,
        state TEXT,
        county VARCHAR,
        latitude REAL,
        longitude REAL,
        POPESTIMATE2023 INTEGER
    )
''')

In [9]:
# Insert the merged data into the table
for index, row in merged_df.iterrows():
    try:
        cursor.execute('''
            INSERT INTO bigfoot_population (report_class, state, county, latitude, longitude, POPESTIMATE2023)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (row['report_class'], row['state'], row['county'], row['latitude'], row['longitude'], row['POPESTIMATE2023']))
    except sqlite3.Error as e:
        print(f"Error inserting row {index}: {e}")

# Commit the changes
conn.commit()

In [10]:
# Query to extract all data from the table
cursor.execute('SELECT * FROM bigfoot_population')

# Fetch all rows and display them
rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 'A', 'Alaska', 'Anchorage County', 61.1199955, -149.7454303, 733406)
(2, 'B', 'Alaska', 'Anchorage County', 61.2175758, -149.8996785, 733406)
(3, 'A', 'Alaska', 'Cordova-McCarthy County', 61.4863889, -142.8863888, 733406)
(4, 'A', 'Alaska', 'Fairbanks County', 64.8400511, -147.7199756, 733406)
(5, 'B', 'Alaska', 'Fairbanks County', 64.8400511, -147.7199756, 733406)
(6, 'B', 'Alaska', 'Matanuska-Susitna County', 61.7697222, -149.3088889, 733406)
(7, 'A', 'Alaska', 'Prince of Wales County', 55.2074214, -132.8275439, 733406)
(8, 'B', 'Alaska', 'Prince of Wales County', 55.4862705, -132.6393743, 733406)
(9, 'A', 'Alaska', 'Prince of Wales County', 55.2074214, -132.8275439, 733406)
(10, 'B', 'Alaska', 'Southeast Fairbanks County', 64.0731336, -141.9384285, 733406)
(11, 'A', 'Alaska', 'Southeast Fairbanks County', 63.3359266, -142.98770150000001, 733406)
(12, 'A', 'Alaska', 'Valdez-Chitina-Whittier County', 62.1081204, -145.5340116, 733406)
(13, 'B', 'Alabama', 'Autauga County', 32.43402

In [11]:
cursor.execute('PRAGMA encoding')
encoding = cursor.fetchone()
print(f"Database encoding: {encoding[0]}")


Database encoding: UTF-8


In [12]:
# Close the database connection
conn.close()